In [8]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_pinecone import PineconeVectorStore
# Correct imports - separate the main Pinecone client from LangChain's wrapper
from pinecone import Pinecone, ServerlessSpec  # Main Pinecone client
from langchain_pinecone import PineconeVectorStore  # LangChain wrapper
load_dotenv()

True

In [3]:
import json
import pickle
from typing import List, Dict, Any
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.documents import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq
from rank_bm25 import BM25Okapi
import re

In [4]:
# ==========================================
# STEP 2: Setup OpenAI Embeddings
# ==========================================

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.getenv("OPENAI_API_KEY")
)

print("OpenAI Embedding Model loaded successfully!")

# Test embeddings
query_result = embeddings.embed_query("Hello world")
print("Embedding dimension:", len(query_result))

OpenAI Embedding Model loaded successfully!
Embedding dimension: 3072


In [9]:
# ==========================================
# STEP 4: Setup Pinecone
# ==========================================

import os
# Set Pinecone API key
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")

# Initialize Pinecone
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
# FIRST: Check what dimension your embeddings actually produce
test_embedding = embeddings.embed_query("test")
actual_dimension = len(test_embedding)
print(f"Actual embedding dimension: {actual_dimension}")

Actual embedding dimension: 3072


In [10]:
# Index settings
index_name = "act"
embedding_dimension =  3072 # text-embedding-3-large dimension

# Create index if it doesn't exist
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=embedding_dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Created new index: {index_name}")
else:
    print(f"Using existing index: {index_name}")


Using existing index: act


In [11]:
# Add this after your Pinecone index setup:
vectorstore = PineconeVectorStore(
    index=pc.Index(index_name),
    embedding=embeddings
)

In [12]:
# Initialize Groq LLM (you can also use OpenAI)
llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.1,
    max_tokens=None
)

# # Initialize Groq LLM (you can also use OpenAI)
# import os
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(
#     api_key=os.getenv("OPENAI_API_KEY"),
#     model_name="gpt-4.1-nano-2025-04-14",
#     temperature=0.1,
#     max_tokens=None
# )

In [13]:
from langchain_core.prompts import ChatPromptTemplate

# ===============================
# System prompt (Legal RAG | BN + EN)
# ===============================
system_prompt = (
    # ---------- Bangla ----------
    "আপনি বাংলাদেশের আইনভিত্তিক একটি লিগ্যাল চ্যাটবট। আপনার জ্ঞানভান্ডারে আইন/অ্যাক্ট, বিধি/রুলস, "
    "অধ্যাদেশ (Ordinance), সংশোধনী (Amendment), প্রজ্ঞাপন/গেজেট/সার্কুলার/নোটিফিকেশন, SRO/GO/RO ইত্যাদি থাকতে পারে। "
    "আপনি RAG চেইনের মাধ্যমে কনটেক্সট (context) পাবেন এবং শুধুমাত্র সেই নথি-উৎসের তথ্যের ভিত্তিতে উত্তর দেবেন—"
    "কল্পনানির্ভর বা অনুমান নির্ভর কিছু বলবেন না।\n\n"
    "নীতিমালা:\n"
    "1) হায়ারার্কি মানুন: আইন → অংশ → অধ্যায় → ধারা → উপধারা → দফা/উপদফা। যে তথ্য উদ্ধৃত করবেন, "
    "   সম্ভব হলে নির্দিষ্ট রেফারেন্স (আইনের নাম, ধারা/উপধারা/দফা নম্বর, গেজেট/প্রজ্ঞাপনের তারিখ ও নম্বর) দিন।\n"
    "2) উত্তরভাষা: ডিফল্টে **বাংলা** ভাষায় উত্তর দিন। শেষে ২–৩ লাইনের একটি **English summary** দিন। "
    "   যদি ব্যবহারকারী স্পষ্টভাবে ইংরেজি চান, তখন সম্পূর্ণ ইংরেজিতেও দিতে পারেন।\n"
    "3) নির্ভুলতা: প্রাসঙ্গিক অংশ না পেলে বলুন—‘প্রদত্ত কনটেক্সটে সুনির্দিষ্ট রেফারেন্স পাওয়া যায়নি’। "
    "   প্রয়োজনে স্পষ্টীকরণ প্রশ্ন করুন (যেমন: আইন/বিধির সাল, ধারা নম্বর)।\n"
    "4) তারিখ ও সংস্করণ: গেজেট/প্রজ্ঞাপনের **সুনির্দিষ্ট তারিখ** (DD-MM-YYYY) এবং কার্যকারিতা (effective date) "
    "   উল্লেখ করুন যদি কনটেক্সটে থাকে। পুরোনো/নতুন—উভয় সংস্করণের পার্থক্য থাকলে তা স্পষ্ট করুন।\n"
    "5) কাঠামো: (ক) সারসংক্ষেপ, (খ) প্রাসঙ্গিক আইনি রেফারেন্স, (গ) বিশ্লেষণ/ব্যাখ্যা, (ঘ) করণীয়/সতর্কতা, "
    "   (ঙ) সোর্স লিস্ট—শিরোনাম/ধারা/গেজেট তারিখসহ।\n"
    "6) দায়স্বীকার: এটি সাধারণ তথ্যগত সহায়তা; এটি আইনি পরামর্শ নয়। প্রয়োজনে আইনজীবীর পরামর্শ নিন।\n\n"
    "নিচের context শুধুমাত্র RAG থেকে এসেছে—উত্তর দেওয়ার সময় এটিই ব্যবহার করুন:\n"
    "{context}\n\n"
    # ---------- English ----------
    "You are a Bangladesh Law Assistant. Your corpus may include Acts/Laws, Rules/Regulations, "
    "Ordinances, Amendments, Gazettes/Notifications/Circulars, and SRO/GO/RO. You will receive "
    "retrieved context via RAG and must answer **only** from that context—avoid speculation.\n\n"
    "Policy:\n"
    "1) Respect hierarchy: Law → Part → Chapter → Section → Subsection → Clause/Sub-clause. "
    "   Cite precise references (act name, section/subsection/clause, gazette number/date) when available.\n"
    "2) Language: Answer **in Bangla by default**, then add a brief **English summary** (2–3 lines). "
    "   If the user explicitly requests English, you may answer fully in English.\n"
    "3) Accuracy: If the context lacks a specific reference, say so and, if helpful, ask for clarifications "
    "(e.g., year/section number).\n"
    "4) Dates & versions: Mention exact Gazette/Notification dates (DD-MM-YYYY) and effective dates if present. "
    "   Distinguish between current and historical versions when applicable.\n"
    "5) Structure: (a) Summary, (b) Relevant legal references, (c) Analysis, (d) Actionable guidance/caveats, "
    "   (e) Source list with titles/sections/dates.\n"
    "6) Disclaimer: Informational assistance only; not legal advice. Recommend consulting a lawyer when needed."
)

# Rebuild prompt with the new system prompt (rest of your code stays the same)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [18]:
#retriever=vectorstore.as_retriever(search_type="mmr", search_kwargs={'k':20})
retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={'k':5})

In [19]:
# --- Build document chain and retrieval chain ---
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("Chains ready: question_answer_chain, rag_chain")


Chains ready: question_answer_chain, rag_chain


In [17]:
# --- Example usage: invoke RAG and print context + answer ---
input_question = "কোম্পানি’ বলতে কোন কোন সত্তা অন্তর্ভুক্ত?"
response = rag_chain.invoke({"input": input_question})
answer = response.get("answer") or response.get("result") or response
print(answer)


## সারসংক্ষেপ
“কোম্পানি” বলতে আইনগত দিক থেকে বিভিন্ন ধরনের সত্তাকে বোঝায়, যা বাংলাদেশের আইন অনুসারে নিগমিত বা প্রতিষ্ঠিত হয়।

## প্রাসঙ্গিক আইনি রেফারেন্স
- **কোম্পানি আইন, ১৯৯৪ (২০০৬ সালের ১৮ নং আইন)**: এই আইনে কোম্পানির সংজ্ঞা দেওয়া হয়েছে।

## বিশ্লেষণ
“কোম্পানি” শব্দটি সাধারণত একটি নিগমিত সত্তাকে বোঝায় যা আইনগত স্বতন্ত্র ব্যক্তিত্বসম্পন্ন। এর মধ্যে বিভিন্ন ধরনের সত্তা যেমন প্রাইভেট লিমিটেড কোম্পানি, পাবলিক লিমিটেড কোম্পানি, পাবলিক কোম্পানি লিমিটেড বাই গ্যারান্টি, প্রাইভেট কোম্পানি লিমিটেড বাই গ্যারান্টি ইত্যাদি অন্তর্ভুক্ত।

## করণীয়/সতর্কতা
কোনো ব্যবসায়িক কার্যক্রম পরিচালনা বা নিবন্ধন করার আগে কোম্পানির ধরন এবং প্রাসঙ্গিক আইনি প্রয়োজনীয়তা সম্পর্কে জানা প্রয়োজন।

## সোর্স লিস্ট
- কোম্পানি আইন, ১৯৯৪ (২০০৬ সালের ১৮ নং আইন), সংশ্লিষ্ট ধারা।

## English Summary
The term "company" refers to various entities incorporated or established under the laws of Bangladesh, including private limited companies, public limited companies, and others.

## Relevant Legal References
- Companie

In [20]:
retrieved_docs = retriever.invoke(input_question)

print("This is the context:")
for idx, doc in enumerate(retrieved_docs, start=1):
    print(f"\nchunk:{idx}")
    print(doc.page_content)
    print("Metadata:", doc.metadata)

print("\n------THIS IS THE ANSWER----")
print(answer)

This is the context:

chunk:1
II of 1877) এ সংজ্ঞায়িত ট্রাস্টও ইহার অন্তর্ভুক্ত হইবে:

তবে শর্ত থাকে যে, যেইক্ষেত্রে ট্রাস্ট কোনো কোম্পানি বা তহবিল গঠনের পূর্বশর্ত অথবা কোম্পানি বা তহবিল পরিচালনার উদ্দেশ্যে গঠিত হইয়া থাকে সেইক্ষেত্রে উক্ত কোম্পানি বা, ক্ষেত্রমত, তহবিল করযোগ্য সন্ত্বা হইবে এবং ট্রাস্ট করযোগ্য সন্ত্বা হিসাবে বিবেচিত হইবে না;

(৩৯) “ট্যাক্স রিকোভারি অফিসার” অর্থ ধারা ৪ এ উল্লিখিত ট্যাক্স রিকোভারি অফিসার হিসাবে নিযুক্ত কোনো ব্যক্তি;

(৪০) “ডিমার্জার” অর্থ অষ্টম তফসিলে সংজ্ঞায়িত ডিমার্জার;

(৪১) “তফসিলি ব্যাংক” অর্থ Bangladesh Bank Order, 1972 (President's Order No. 127 of 1972) এর article 2(j) - তে সংজ্ঞায়িত কোনো তফসিলি ব্যাংক;

(৪২) “তহবিল” অর্থ আপাতত বলবৎ কোনো আইন বা বিধির অধীন সৃষ্ট বা স্বীকৃত কোনো তহবিল;

(৪৩) “দাতব্য উদ্দেশ্য” অর্থ—

(ক) দরিদ্রের জন্য ত্রাণ, শিক্ষা ত্রাণ, চিকিৎসা ত্রাণ; এবং

(খ) সাধারণ জন-উপযোগের কোনো উদ্দেশ্যের উন্নতি বা প্রসার,

তবে সাধারণ জন-উপযোগের কোনো উদ্দেশ্যের উন্নতি বা প্রসার দাতব্য উদ্দেশ্য বলিয়া বিবেচিত হইবে না-

(অ) যদি না উহা জাতীয় 